In [12]:
import pandas as pd
import re

In [3]:
org_ntw = pd.read_csv('../Nov.25__smartlog.scrapper/smlog_data/네트워크_merged.csv')
org_md = pd.read_csv('../Nov.25__smartlog.scrapper/smlog_data/미디어_merged.csv')
org_site = pd.read_csv('../Nov.25__smartlog.scrapper/smlog_data/사이트_merged.csv')
org_kwd = pd.read_csv('../Nov.25__smartlog.scrapper/smlog_data/키워드_merged.csv')

In [15]:
ntw_columns = ['media','network','totalClick','validClick','churnRate','avgSesstionTime','conv_ask','conv_join','conv_order','conv_rev','date']
kwd_columns = ['media','keyword','totalClick','validClick','churnRate','avgSesstionTime','conv_ask','conv_join','conv_order','conv_rev','date']
site_columns = ['media','site','totalClick','validClick','churnRate','avgSesstionTime','conv_ask','conv_join','conv_order','conv_rev','date']
md_columns = ['media','totalClick','validClick','churnRate','avgSesstionTime','conv_ask','conv_join','conv_order','conv_rev','date']

org_ntw.columns = ntw_columns
org_kwd.columns = kwd_columns
org_site.columns = site_columns
org_md.columns = md_columns


# 예시: churnRate, conv_rev에서 %와 ₩를 제거하고 int로 변환하도록 apply 함수 사용
for df in [org_ntw, org_kwd, org_site, org_md]:
    if 'churnRate' in df.columns:
        df['churnRate'] = df['churnRate'].apply(lambda x: int(str(x).replace('%', '')))
    if 'conv_rev' in df.columns:
        df['conv_rev'] = df['conv_rev'].apply(lambda x: int(str(x).replace('₩', '').replace(',', '')))
    df['date'] = pd.to_datetime(df['date'])

org_ntw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162 entries, 0 to 161
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   media            162 non-null    object        
 1   network          162 non-null    object        
 2   totalClick       162 non-null    int64         
 3   validClick       162 non-null    int64         
 4   churnRate        162 non-null    int64         
 5   avgSesstionTime  162 non-null    object        
 6   conv_ask         162 non-null    int64         
 7   conv_join        162 non-null    int64         
 8   conv_order       162 non-null    int64         
 9   conv_rev         162 non-null    int64         
 10  date             162 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(7), object(3)
memory usage: 14.1+ KB


,media,network,totalClick,validClick,churnRate,avgSesstionTime,conv_ask,conv_join,conv_order,conv_rev,date
0,구글,Google 검색,3641,2838,97%,0:00:49,71,186,458,₩275,2025-11-05
1,네이버,파워링크,1204,1026,81%,0:03:01,40,21,50,₩9,2025-11-05
2,구글,Google 검색,3641,2838,97%,0:00:49,71,186,458,₩275,2025-11-06
3,네이버,파워링크,1204,1026,81%,0:03:01,40,21,50,₩9,2025-11-06
4,구글,Google 검색,3312,2564,97%,0:00:49,65,161,412,₩248,2025-08-18
...,...,...,...,...,...,...,...,...,...,...,...
157,네이버,파워링크,1124,959,80%,0:03:03,36,19,45,₩9,2025-11-02
158,구글,Google 검색,3312,2564,97%,0:00:49,65,161,412,₩248,2025-11-03
159,네이버,파워링크,1124,959,80%,0:03:03,36,19,45,₩9,2025-11-03
160,구글,Google 검색,3312,2564,97%,0:00:49,65,161,412,₩248,2025-11-04


In [7]:
org_ntw.columns

Index(['매체', '네트워크', '광고클릭', '이탈률', '평균체류시간', '전환', 'Column_6', 'Column_7',
       'Column_8', 'Column_9', 'date'],
      dtype='object')